In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.nn.modules.loss import BCEWithLogitsLoss, BCELoss
import rasterio
import numpy as np
import matplotlib
import os
import glob
import matplotlib.pyplot as plt
import cv2
from IPython.core.pylabtools import figsize
from Model.load_data import CustomGeoDataset
from Model.unet import unet
from Model.TransUnet import TransUNet
import segmentation_models_pytorch as smp

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
#DEVICE = 'cpu'
print("[INFO] training using {} ...".format(DEVICE))

[INFO] training using cuda ...


In [2]:
Image_Drive = "./Data/train/images"
Label_Drive = "./Data/train/labels"

#augmented data
#Image_aug = "./Data/Transformed_images"
#Label_aug = "./Data/Transformed_labels"

image_path = glob.glob(os.path.join(Image_Drive, "*.tif"))
label_path = glob.glob(os.path.join(Label_Drive, "*.tif"))


image_size = len(image_path)
label_size = len(label_path)

print(image_size)
print(label_size)

2457
2457


aug_image_path = glob.glob(os.path.join(Image_aug, "*.tif"))
aug_label_path = glob.glob(os.path.join(Label_aug, "*.tif"))

image_path = image_path + aug_image_path
label_path = label_path + aug_label_path

image_size = len(image_path)
label_size = len(label_path)

print(image_size)
print(label_size)

In [8]:
# Instantiate the custom dataset with transformations
train_data = CustomGeoDataset(image_path, label_path)
print(len(train_data))

split_size = 0.80
train_size = int(split_size * len(train_data))
test_size = len(train_data) - train_size
train_data, test_data = torch.utils.data.random_split(train_data, [train_size, test_size])

#dataloader
trainloader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=8)
testloader = torch.utils.data.DataLoader(test_data, shuffle = True, batch_size=8)

print(len(trainloader))
print(len(testloader))

2457
246
62


In [4]:
def make_train_step(model, optimizer, loss_fn):
    def train_step(x, y):
        # Ensure x has the correct shape
        if x.dim() == 3:
            x = x.unsqueeze(1)  # Add channel dimension
        
        # Make prediction
        yhat = model(x)
        # Enter train mode
        model.train()
        # Compute loss
        loss = loss_fn(yhat, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        return loss
    return train_step

# model unetransformer
#modelunetplusplus = smp.UnetPlusPlus(in_channels=4, encoder_depth=5, classes=1,
#    activation=None,
#    encoder_weights="imagenet",
#    encoder_name="efficientnet-b4",
#    decoder_attention_type="scse")

modelunetplusplus = smp.UnetPlusPlus(in_channels=4, encoder_depth=5, classes=1, encoder_name="efficientnet-b4", decoder_attention_type="scse")


# Define the U-Net model
model_unet = smp.Unet(
    encoder_name="efficientnet-b4",     # Choose the encoder, e.g., efficientnet-b4
    encoder_depth=5,                    # Choose depth for encoder
    in_channels=4,                      # Model input channels (1 for grayscale, 3 for RGB, etc.)
    classes=1,                          # Model output channels (number of classes)
    decoder_attention_type="scse"       # Optional, add SCSE attention module to decoder
)

# Define the DeepLabV3+ model
model_deeplabv3plus = smp.DeepLabV3Plus(
    encoder_name="efficientnet-b4",     # Choose the encoder, e.g., efficientnet-b4
    encoder_depth=5,                    # Choose depth for encoder
    in_channels=4,                      # Model input channels (1 for grayscale, 3 for RGB, etc.)
    classes=1                           # Model output channels (number of classes)
)

# Define the FPN model
model_fpn = smp.FPN(
    encoder_name="efficientnet-b4",  # Choose the encoder, e.g., efficientnet-b4
    encoder_depth=5,                 # Choose depth for encoder
    in_channels=4,                   # Model input channels (1 for grayscale, 3 for RGB, etc.)
    classes=1                        # Model output channels (number of classes)
)

# Define the PSPNet model
model_pspnet = smp.PSPNet(
    encoder_name="efficientnet-b4",  # Choose the encoder, e.g., efficientnet-b4
    encoder_depth=5,                 # Choose depth for encoder
    in_channels=4,                   # Model input channels (1 for grayscale, 3 for RGB, etc.)
    classes=1                        # Model output channels (number of classes)
)

# Define the PAN model
model_pan = smp.PAN(
    encoder_name="efficientnet-b4",  # Choose the encoder, e.g., efficientnet-b4
    in_channels=4,                   # Model input channels (1 for grayscale, 3 for RGB, etc.)
    classes=1                       # Model output channels (number of classes)
)

# Define the PAN model
model_manet = smp.MAnet(
    encoder_name="efficientnet-b4",  # Choose the encoder, e.g., efficientnet-b4
    encoder_depth=5,                 # Choose depth for encoder
    in_channels=4,                   # Model input channels (1 for grayscale, 3 for RGB, etc.)
    classes=1                        # Model output channels (number of classes)
)

print(modelunetplusplus)

modelunetplusplus = modelunetplusplus.to(DEVICE)
model_unet = model_unet.to(DEVICE)
model_pan = model_pan.to(DEVICE)
model_pspnet = model_pspnet.to(DEVICE)
model_fpn = model_fpn.to(DEVICE)
model_deeplabv3plus = model_deeplabv3plus.to(DEVICE)
model_manet = model_manet.to(DEVICE)

In [ ]:
from torch.optim import SGD

model_transunet = TransUNet(img_dim=128,
                          in_channels=4,
                          out_channels=128,
                          head_num=4,
                          mlp_dim=512,
                          block_num=8,
                          patch_dim=16,
                          class_num=1).to(DEVICE)

model_list = {'model_transunet': model_transunet}

# define training hyperparameters
INIT_LR = 1e-4
epochs = 250

# initialize our optimizer and loss function
optimizer = SGD(model_transunet.parameters(), lr=INIT_LR)
lossfn = BCEWithLogitsLoss()

#train step
train_step = make_train_step(model_transunet, optimizer, lossfn)
print(train_step)

<function make_train_step.<locals>.train_step at 0x0000025429E29300>


In [10]:
from tqdm import tqdm
from torchmetrics import JaccardIndex

def train(model, trainloader, optimizer, loss_fn, device):
    model.train()
    train_step = make_train_step(model, optimizer, loss_fn)
    epoch_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    jaccard = JaccardIndex(task='binary').to(DEVICE)

    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        x_batch, y_batch = data
        x_batch = x_batch.float().to(device)  # Ensure correct type
        y_batch = y_batch.float().to(device)

        # Forward pass and loss calculation
        loss = train_step(x_batch, y_batch)
        epoch_loss += loss.item()

        # Predictions
        predictions = (model(x_batch) > 0.5).float()  # Binary thresholding at 0.5

        # Accuracy calculation
        correct_predictions += torch.sum(predictions == y_batch).item()
        total_predictions += (y_batch.size(0) * y_batch.size(1) * y_batch.size(2) * y_batch.size(3))

        # jaccard index (IoU) calculation
        jaccard.update(predictions, y_batch)

    accuracy = (correct_predictions/total_predictions) * 100  # Convert to percentage
    average_loss = epoch_loss / len(trainloader)
    iou = jaccard.compute()

    return average_loss, accuracy, iou

def evaluate(model, testloader, loss_fn, device):
    model.eval()
    epoch_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    jaccard = JaccardIndex(task='binary').to(DEVICE)

    with torch.no_grad():
        for x_batch, y_batch in testloader:
            x_batch = x_batch.float().to(device)
            y_batch = y_batch.float().to(device)

            # Forward pass and loss calculation
            yhat = model(x_batch)
            loss = loss_fn(yhat, y_batch)
            epoch_loss += loss.item()

            # Predictions
            predicted_classes = (yhat > 0.5).float() # Round to get binary predictions
            correct_predictions += torch.sum(predicted_classes == y_batch).item()
            total_predictions += (y_batch.size(0) * y_batch.size(1) * y_batch.size(2) * y_batch.size(3))

            #jaccard calculation
            jaccard.update(predicted_classes, y_batch)

    accuracy = (correct_predictions / total_predictions) * 100  # Convert to percentage
    average_loss = epoch_loss / len(testloader)
    iou = jaccard.compute()

    return average_loss, accuracy, iou


In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        Args:
            patience (int): Berapa banyak epoch untuk menunggu sebelum berhenti ketika tidak ada peningkatan.
            delta (float): Minimum perubahan pada monitored metric untuk dianggap peningkatan.
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.best_model_wts = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model_wts = model.state_dict()
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
best_loss3 = float('inf')
best_model_wts3 = None

train_losses3 = []
train_accuracies3 = []
test_losses3 = []
test_accuracies3 = []
train_ious3 = []
test_ious3 = []

patience = 10
# Initialize EarlyStopping
early_stopping = EarlyStopping(patience=patience)

for epoch in range(epochs):
    train_loss, train_accuracy, train_iou = train(model_transunet, trainloader, optimizer, lossfn, DEVICE)
    test_loss, test_accuracy, test_iou = evaluate(model_transunet, testloader, lossfn, DEVICE)

    train_losses3.append(train_loss)
    train_accuracies3.append(train_accuracy)
    test_losses3.append(test_loss)
    test_accuracies3.append(test_accuracy)
    train_ious3.append(train_iou)
    test_ious3.append(test_iou)

    print(f'Epoch {epoch + 1}/{epochs}: Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train IOU: {train_iou:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test IOU: {test_iou:.4f}')

    # Early stopping check
    early_stopping(test_loss, model_transunet)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Save the best model
    if test_loss < best_loss3:
        best_loss3 = test_loss

        best_model_wts3 = model_transunet.state_dict()

model_transunet.load_state_dict(best_model_wts3)


  0%|          | 0/246 [00:00<?, ?it/s]c:\Users\sanji\anaconda3\Lib\site-packages\rasterio\__init__.py:317: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
100%|██████████| 246/246 [01:34<00:00,  2.61it/s]


Epoch 1/250: Train Loss: 0.6446, Train Accuracy: 58.0831, Train IOU: 0.1778, Test Loss: 0.5715, Test Accuracy: 65.4166, Test IOU: 0.3530


100%|██████████| 246/246 [01:40<00:00,  2.44it/s]


Epoch 2/250: Train Loss: 0.5971, Train Accuracy: 63.7264, Train IOU: 0.2899, Test Loss: 0.5547, Test Accuracy: 67.1409, Test IOU: 0.3841


100%|██████████| 246/246 [01:39<00:00,  2.48it/s]


...

100%|██████████| 246/246 [01:02<00:00,  3.91it/s]


Epoch 249/250: Train Loss: 0.4074, Train Accuracy: 80.4254, Train IOU: 0.6346, Test Loss: 0.3708, Test Accuracy: 82.1361, Test IOU: 0.6656


100%|██████████| 246/246 [01:32<00:00,  2.65it/s]


Epoch 250/250: Train Loss: 0.4117, Train Accuracy: 80.2158, Train IOU: 0.6320, Test Loss: 0.3974, Test Accuracy: 80.4900, Test IOU: 0.6365


<All keys matched successfully>

In [12]:
torch.save(model_transunet, 'umodel_transunet_data_dne.pth')
#load model
#modelunet = torch.load('modelunet.pth')

In [13]:
import pandas as pd

trls = train_losses3
print(trls)
tracc = train_accuracies3
print(tracc)
tsls = test_losses3
print(tsls)
tsacc = test_accuracies3
print(tsacc)
triou = [t1.cpu().numpy() for t1 in train_ious3]
print(triou)
tsiou = [t2.cpu().numpy() for t2 in test_ious3]
print(tsiou)

# Create a dictionary to form the DataFrame
data = {
    'train_losses': trls,
    'train_accuracies': tracc,
    'test_losses': tsls,
    'test_accuracies': tsacc,
    'train_ious': triou,
    'test_ious': tsiou
}

# Create the DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('performance_metrics_transunet_data_dne.csv', index=False)

[0.6446468922180858, 0.5971033699144193, 0.5839592535805896, 0.5771672795943128, 0.5684436761993703, 0.5664674373177009, 0.5628182602122547, 0.5547764962766228, 0.5535727269281217, 0.5499348805202702, 0.547441281802286, 0.5510017974832193, 0.5455704828103384, 0.5457512057893644, 0.5367391831506558, 0.5382124375763947, 0.5325114220623078, 0.5363763517480555, 0.5287909076465824, 0.5284787427361418, 0.5275722439211559, 0.524252770514023, 0.5163003750206009, 0.5160919934268889, 0.5086610388707339, 0.5227021024963721, 0.5089108403620681, 0.5102320908288646, 0.5049764406148011, 0.5046620521603561, 0.5043712781696785, 0.5006334722284379, 0.5015924782530079, 0.4987028465280688, 0.4981575776648715, 0.4973074538194067, 0.494161102103024, 0.491422487225959, 0.49139066246466906, 0.487242775113602, 0.4919679710050909, 0.4889398713906606, 0.48632539905668276, 0.4877616965431508, 0.49091873481506254, 0.48307446202611537, 0.4836291065060995, 0.48342766267497367, 0.4843429217978222, 0.4810816329426882,

In [ ]:
import rasterio
from rasterio.transform import from_origin
from PIL import Image
import numpy as np

out_folder_img = './Data/Test_Results/Image'
out_folder_label = './Data/Test_Results/Label'

def save_image_rasterio(image, filepath):
    # image is expected to be of shape (channels, height, width)
    if image.ndim == 4 and image.shape[0] == 1:
        image = image[0]  # Remove the channel dimension if it's single channel

    height, width = image.shape[-2], image.shape[-1]
    transform = from_origin(0, 0, 1, 1)  # Modify as per your coordinate reference system

    with rasterio.open(
        filepath,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1 if image.ndim == 2 else image.shape[0],  # Set the count to number of channels
        dtype=image.dtype,
        crs='+proj=latlong',
        transform=transform,
    ) as dst:
        if image.ndim == 2:
            dst.write(image, 1)  # Assuming the image is single channel (grayscale)
        else:
            for band in range(image.shape[0]):
                dst.write(image[band], band + 1)  # Write each channel separately

def make(model, testloader, device):
    model.eval()
    predicted_images = []  # List to store predicted images
    input_images = []  # List to store input images

    with torch.no_grad():
        for batch_idx, (x_batch, y_batch) in enumerate(testloader):
            x_batch = x_batch.float().to(device)
            y_batch = y_batch.float().to(device)

            # Forward pass and loss calculation
            yhat = model(x_batch)
  
            # Predictions
            predicted_classes = (yhat > 0.5).float() # Round to get binary predictions
            
            # Saving predicted images
            for i in range(predicted_classes.size(0)):  # Iterate over the batch dimension
                predicted_image = predicted_classes[i].cpu().numpy()
                input_image = x_batch[i].cpu().numpy()
                save_image_rasterio(predicted_image, f"{out_folder_label}/prediction_{batch_idx}_{i}.tif")
                save_image_rasterio(input_image, f"{out_folder_img}/input_{batch_idx}_{i}.tif")
                #save_image_png(predicted_image, f"{out_folder_img}/prediction_{batch_idx}_{i}.png")
                #save_image_png(input_image, f"{out_folder_label}/input_{batch_idx}_{i}.png")
                predicted_images.append(predicted_image)  # Store the predicted image in the list
                input_images.append(input_image)  # Store the input image in the list

    return input_images, predicted_image


def save_image_png(image, filepath):
    # image is expected to be of shape (channels, height, width)
    if image.ndim == 3 and image.shape[0] == 1:
        image = image[0]  # Remove the channel dimension if it's single channel

    if image.ndim == 2:  # Single channel image
        image = (image * 255).astype(np.uint8)  # Convert to uint8
        Image.fromarray(image).save(filepath)
    else:  # Multi-channel image
        if image.shape[0] == 4:  # Drop the fourth channel if present
            image = image[:3, :, :]
        image = np.transpose(image, (1, 2, 0))  # Convert from (channels, height, width) to (height, width, channels)
        image = (image * 65535).astype(np.uint8)  # Convert to uint8
        Image.fromarray(image).save(filepath)